# Generative Adversarial Network

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [ ]:
os.chdir("./drive/My Drive/Colab Notebooks/pytorch")

In [ ]:
# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 100 #originally 200
batch_size = 100
sample_dir = 'samples'

In [ ]:
# Create a directory if it doesnt exist
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [ ]:
# Image processing
# transform = transforms.Compose([
#                transforms.ToTensor(),
#                transforms.Normalize(mean=(0.5, 0.5, 0.5), # 3 for RGB chnnales
#                                     std = (0.5, 0.5, 0.5))   
# ])
transform = transforms.Compose([
                 transforms.ToTensor(),
                 transforms.Normalize(mean=[0.5], # 1 for greyscale channels
                                      std=[0.5])
])

In [ ]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

In [ ]:
# Data Loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size,
                                          shuffle=True)

In [ ]:
# Discriminator
D = nn.Sequential(
      nn.Linear(image_size, hidden_size),
      nn.LeakyReLU(0.2), # negative slope value = 0.2
      nn.Linear(hidden_size, hidden_size),
      nn.LeakyReLU(0.2),
      nn.Linear(hidden_size, 1),
      nn.Sigmoid()
)

In [ ]:
# Generator
G = nn.Sequential(
      nn.Linear(latent_size, hidden_size),
      nn.ReLU(),
      nn.Linear(hidden_size, hidden_size),
      nn.ReLU(),
      nn.Linear(hidden_size, image_size),
      nn.Tanh()
)

In [ ]:
# Device setting
D = D.to(device)
G = G.to(device)

In [ ]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [ ]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [ ]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)

        # create the labels which will be used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        ######## Train the discriminator
        #######################################

        # compute BCE_Loss using real images where 
        # BCE_Loss(x, y): -y * log(D(x)) - (1-y) * log(1-D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()

        ######## Train the generator
        ##########################################

        # compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)

        # train G to maximize log(D(G(z))) instead of minimising log(1-D(G(z)))
        # details in the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)

        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()

        if (i+1) % 200 == 0:
            print("Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss:{:.4f}, D(x):{:.2f}, D(G(z)): {:.2f}".format(
                  epoch + 1, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(),
                  real_score.mean().item(), fake_score.mean().item()))

    # save real image
    if (epoch) == 0:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))   
    
    # save sampled images
    if (epoch % 10) == 0:
      fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
      save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))




Epoch [1/100], Step [200/600], d_loss: 0.0471, g_loss:4.0262, D(x):0.99, D(G(z)): 0.04
Epoch [1/100], Step [400/600], d_loss: 0.0544, g_loss:5.4495, D(x):0.99, D(G(z)): 0.04
Epoch [1/100], Step [600/600], d_loss: 0.0477, g_loss:5.1277, D(x):0.98, D(G(z)): 0.02
Epoch [2/100], Step [200/600], d_loss: 0.0557, g_loss:4.8655, D(x):0.99, D(G(z)): 0.03
Epoch [2/100], Step [400/600], d_loss: 0.2376, g_loss:4.7114, D(x):0.91, D(G(z)): 0.09
Epoch [2/100], Step [600/600], d_loss: 0.1379, g_loss:4.7905, D(x):0.94, D(G(z)): 0.04
Epoch [3/100], Step [200/600], d_loss: 0.2502, g_loss:4.1708, D(x):0.88, D(G(z)): 0.05
Epoch [3/100], Step [400/600], d_loss: 0.0854, g_loss:5.0844, D(x):0.97, D(G(z)): 0.04
Epoch [3/100], Step [600/600], d_loss: 0.7766, g_loss:3.3485, D(x):0.77, D(G(z)): 0.17
Epoch [4/100], Step [200/600], d_loss: 0.2641, g_loss:3.0817, D(x):0.95, D(G(z)): 0.18
Epoch [4/100], Step [400/600], d_loss: 0.5851, g_loss:2.7562, D(x):0.87, D(G(z)): 0.29
Epoch [4/100], Step [600/600], d_loss: 0.78

In [ ]:
# save the model checkpoints
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')